In [ ]:
from abc import ABC, abstractmethod
from collections import Counter
from itertools import combinations
import random

In [ ]:
class Utils:
    def check_counts(self, string):
        for item, value in Counter(string).items():
            if value%2 == 1:
                return False
        return True

    def generate_substrings(self, string, masterlist):
        if string not in masterlist:
            if self.check_counts(string):
                masterlist.append(string)
        if len(string) < 2:
            return
        else:
            self.generate_substrings(string[1:], masterlist)
            self.generate_substrings(string[:-1], masterlist)

    def evaluate_combination(self, a, b, substring):
        for i in range(len(a)):
            if substring[a[i]] != substring[b[i]]:
                return False
        return True

    def check_substring_for_twins(self, substring):
        length = len(substring)
        indices = set(range(length))

        for a in combinations(indices, int(length/2)):
            b = tuple(indices - set(a))
            if self.evaluate_combination(a, b, substring):
                #print(f"Found twins in {substring} with indices{a} and {b}")
                return [substring, a, b]
        return None
    
    def subsample(self, iterable, rate):
        if rate > 0.9:
            rate = 0.9
        return random.sample(iterable, k = int((1-rate)*len(iterable)))
    
    def generate_possible_positions(self, word):
        return [x for x in range(len(word)+1)]
    
    def deduplicate(self, iterable):
        d = {}
        for x in iterable:
            d[x] = None
        return list(d.keys())
    

In [ ]:
class WordLookup:
    def __init__(self):
        self.lookup = {}
        
    def was_evaluated(self, word):
        return word in self.lookup.keys()
    
    def add_evaluated_and_combinations(self, string, twins):
        start = string.find(twins)
        end = start + len(twins)
        for i in range(0, start + 1):
            for j in range(end, len(string)):
                self.add_evaluated(string[i:j+1], True)
    
    def add_evaluated(self, word, result):
        self.lookup[word] = result
        
    def has_twins(self, word):
        if not self.was_evaluated(word):
            raise IndexError
        else:
            return self.lookup[word]

In [ ]:
class GameState:
    
    def __init__(self, config):
        self.current_string = ""
        self.current_position = 0
        self.alphabet = config['alphabet']
        self.winner = None
        self.playing = True
        self.lookup  = WordLookup()
        self.n_ahead = config['n_steps_ahead']
        self.sample_rate_per_step['sample_rate_per_step']
        
        
    def get_current_string(self):
        return self.current_string
    
    def get_alphabet(self):
        return self.alphabet
    
    def get_current_position(self):
        return self.current_position
    
    def get_winner(self):
        return self.winner
    
    def get_string_length(self):
        return len(self.current_string)
    
    def set_current_position(self, position):
        self.current_position = position
        
    def set_winner(self, player):
        self.winner = player
        
    def set_current_string(self, string):
        self.current_string = string
    
    def is_playing(self):
        return self.playing
    
    def is_not_playing(self):
        self.playing = False
    
    def update_current_string(self, char):
        self.current_string = self.current_string[:self.current_position] + char + self.current_string[self.current_position:]

In [ ]:
class Player(ABC):
    
    @abstractmethod
    def move(self):
        pass

In [ ]:
class Human(Player):
    def move(self, game_state):
        self.print_possible_positions(game_state)

        position = None
        while position == None:
            try:
                position = int(input("Choose position: "))
                if not self.check_validity(position, game_state):
                    position = None
                    print("Invalid position!")
            except:
                position = None
                print("Invalid input!")
            
        game_state.set_current_position(position)
        
    def check_validity(self, position, game_state):
        current_string = game_state.get_current_string()
        return position >= 0 and position <= len(current_string)
    
    def print_possible_positions(self, game_state):
        current_string = game_state.get_current_string()
        print_string = "_"
        for letter in current_string:
            print_string += letter
            print_string += "_"
        
        number_string = ""
        num = 0
        for character in print_string:
            if character == "_":
                number_string += str(num)
                num += 1
            else:
                number_string += " "
        print(f"{print_string}\n{number_string}")

In [ ]:
class AI(Player):
    def __init__(self, utils):
        self.utils = utils
        
    def move(self, game_state):
        alphabet = game_state.get_alphabet()
        current_string = game_state.get_current_string()
        current_position = game_state.get_current_position()
        
        char = self.find_best_letter(current_string, current_position, alphabet, game_state)
        game_state.update_current_string(char)
        
    def evaluate_single_string(self, string, game_state):
        return self.check_for_twins(string, game_state)
    
    def check_for_twins(self, string, game_state):
        if game_state.lookup.was_evaluated(string):
            return game_state.lookup.has_twins(string)
        else:
            substrings = []
            self.utils.generate_substrings(string, substrings)
            for substring in self.utils.deduplicate(reversed(substrings)):
                if game_state.lookup.was_evaluated(substring):
                    if game_state.lookup.has_twins(substring):
                        return True
                twins_check = self.utils.check_substring_for_twins(substring)
                if twins_check:
                    pos1 = twins_check[1]
                    pos2 = twins_check[2]
                    if len(pos1) > 1:
                        twins = substring[pos1[0]:pos2[1]+1]
                    else:
                        twins = substring[pos1[0]:pos2[0]+1]
                    
                    game_state.lookup.add_evaluated_and_combinations(string, twins)
                    return True
                else:
                    game_state.lookup.add_evaluated(substring, False)
            return False
    
    def generate_strings(self, current_string, current_position, alhpabet):
        beggining = current_string[:current_position]
        end = current_string[current_position:]
        strings = [f"{beggining}{char}{end}" for char in alhpabet]
        
        return strings
    

    def generate_strings_for_lookahead(self, candidate, alphabet, n_ahead):
        to_evaluate_per_level = {}
        candidates = [candidate]
        for level in range(0, n_ahead):
            to_evaluate = []
            for candidate in candidates:
                for pos in self.utils.generate_possible_positions(candidate):
                    strings = self.generate_strings(candidate, pos, alphabet)
                    to_evaluate.extend(strings)
            candidates = to_evaluate
            to_evaluate_per_level[level] = to_evaluate
        
        return to_evaluate_per_level
    
    def evaluate_sample(self, sample, game_state):
        twins = 0
        not_twins = 0
        
        for candidate in sample:
            if game_state.lookup.was_evaluated(candidate):
                if game_state.lookup.has_twins(candidate):
                    twins += 1
                else:
                    not_twins += 1
            else:
                check_result = self.check_for_twins(candidate, game_state)
                if check_result:
                    game_state.lookup.add_evaluated(candidate, True)
                    twins += 1
                else:
                    game_state.lookup.add_evaluated(candidate, False)
                    not_twins += 1
        return twins/len(sample)
    
    def calculate_prob_of_twins(self, candidate, game_state, n_ahead, alphabet, sample_rate_per_step = 0.10):
        sample_rate_per_step = game_state.sample_rate_per_step
        lookahead_candidates = self.generate_strings_for_lookahead(candidate, alphabet, n_ahead)
        
        probs_per_level = {}
        for level, candidates in lookahead_candidates.items():
            if level > 0:
                sample = self.utils.subsample(candidates, sample_rate_per_step * (len(candidate)-1))
            else:
                sample = candidates
            probs = self.evaluate_sample(sample, game_state)
            probs_per_level[level] = probs
            
        return probs_per_level
            
    def calculate_risks_from_probs(self, probs, n_ahead):
        risks = {}
        for candidate, probabilities in probs.items():
            risk = []
            for level, prob in probabilities.items():
                risk.append(prob * (n_ahead - level))
            risks[candidate] = sum(risk)
            
        return risks
    
    def find_best_letter(self, current_string, current_position, alphabet, game_state, n_ahead = 3):
        n_ahead = game_state.n_ahead
        candidates = self.generate_strings(current_string, current_position, alphabet)
        
        current_possibilities = []
        has_twins = []
        for candidate in candidates:
            #print(candidate)
            if not self.evaluate_single_string(candidate, game_state):
                current_possibilities.append(candidate)
            else:
                has_twins.append(candidate)
        
        if n_ahead and current_possibilities:
            probs = {}
            for candidate in current_possibilities:
                probs[candidate] = self.calculate_prob_of_twins(candidate, game_state, n_ahead, alphabet)
            #print(probs, has_twins)
            risks = self.calculate_risks_from_probs(probs, n_ahead)
            print(f"AI: detected twins in following candidate strings: {has_twins}")
            for cand, risk in risks.items():
                print(f"AI: candidate: \"{cand}\" risk_score:{risk}")
            return min(risks, key=risks.get)[current_position]
        
        elif current_possibilities:        
            return current_possibilities[0][current_position]
            
        print('AI: No wining option!')
        return alphabet[0]

    

In [ ]:
class Game:
    def __init__(self, config):
        self.utils = Utils()
        self.player1 = Human()
        self.player2 = AI(self.utils)
        self.config = config
        self.game_state = GameState(config)
        
    def play(self):
        while(self.game_state.is_playing()):
            self.game_state.set_winner(None)
            #self.game_state.set_current_string('abca')
            while(self.game_state.get_string_length() < self.config['n']):
                print('Player 1:')
                self.player1.move(self.game_state)
                print('Player 2:')
                self.player2.move(self.game_state)
                if self.evaluate_string(self.game_state.get_current_string()):
                    self.game_state.set_winner('Player1')
                    break
            if not self.game_state.get_winner():
                self.game_state.set_winner('Player2')
            print(f"{self.game_state.get_winner()} wins!")
            
            if input("Play again? y/n: ") == "n":
                self.game_state.is_not_playing()
                
    def check_for_twins(self, string):
        substrings = []
        self.utils.generate_substrings(string, substrings)
        for substring in set(substrings):
            if self.utils.check_substring_for_twins(substring):
                return True
        return False
                
    def evaluate_string(self, string):
        return self.check_for_twins(string)
                

In [ ]:
simple_config = {'n': 7, 'alphabet': ['a','b','c','d'], n_steps_ahead = 3, sample_rate_per_step = 0.10}

In [ ]:
gs = GameState(simple_config)

In [ ]:
game = Game(simple_config)

In [ ]:
game.play()

In [11]:
simple_config = {'n': 7, 'alphabet': ['a','b','c','d'], n_steps_ahead = 3, sample_rate_per_step = 0.10}

SyntaxError: invalid syntax (239626551.py, line 1)

In [28]:
gs = GameState(simple_config)

In [29]:
game = Game(simple_config)

In [30]:
game.play()

Player 1:
_
0


Choose position:  0


Player 2:
AI: detected twins in following candidate strings: []
AI: candidate: "a" risk_score:2.25
AI: candidate: "b" risk_score:2.25
AI: candidate: "c" risk_score:2.25
AI: candidate: "d" risk_score:2.25
Player 1:
_a_
0 1


Choose position:  1


Player 2:
AI: detected twins in following candidate strings: ['aa']
AI: candidate: "ab" risk_score:2.8810158268733854
AI: candidate: "ac" risk_score:2.8228762919896644
AI: candidate: "ad" risk_score:2.8266378660637383
Player 1:
_a_c_
0 1 2


...

Choose position:  0


Player 2:
AI: detected twins in following candidate strings: ['ccbadc']
AI: candidate: "acbadc" risk_score:3.8334573412698414
AI: candidate: "bcbadc" risk_score:4.160590277777777
AI: candidate: "dcbadc" risk_score:3.8348834325396823
Player 1:
_a_c_b_a_d_c_
0 1 2 3 4 5 6


Choose position:  0


Player 2:
AI: detected twins in following candidate strings: ['aacbadc', 'bacbadc']
AI: candidate: "cacbadc" risk_score:4.121387613732063
AI: candidate: "dacbadc" risk_score:3.5942722032586873
Player2 wins!


Play again? y/n:  n
